<a href="https://colab.research.google.com/github/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiny Object Detection : Normalized Gaussian Wasserstein Distance (NWD)

This is an user implementation of Normalized Gaussian Wasserstein Distance for Tiny Object Detection(under review, https://arxiv.org/abs/2110.13389). This implementation includes 

- train Faster R-CNN on VisDrone2019 and AI-TOD using NWD.

Let's start!


In [103]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [104]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
# it may take some time
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.5.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
# you can ignore when restart rutime warning appear
!pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.5.0/index.html
Cloning into 'mmdetection'...
remote: Enumerating objects: 22371, done.
remote: Total 22371 (delta 0), reused 0 (delta 0), pack-reused 22371
Receiving objects: 100% (22371/22371), 25.47 MiB | 31.96 MiB/s, done.
Resolving deltas: 100% (15659/15659), done.
/content/mmdetection/mmdetection
Obtaining file:///content/mmdetection/mmdetection
  Attempting uninstall: mmdet
    Found existing installation: mmdet 2.19.1
    Can't uninstall 'mmdet'. No files were found to uninstall.
  Running setup.py develop for mmdet


In [105]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

from glob import glob
import os



1.5.1+cu101 True
2.19.1
10.1
GCC 7.3


### Dataset-1 : VisDrone2019

In [ ]:
# If you have AI-TOD datset in your Google Drive, 
# simply connect to the Google Drive folder 

download_visdrone = False
if download_visdrone :
  # download, decompress the data
  # not completed yet.. 
  !wget https://drive.google.com/file/d/1a2oHjcEcwXP8oUF95qiwrqzACb2YlUhn/view?usp=sharing
else: 
  from google.colab import drive
  drive.mount('/content/drive/')
  data_root_dir = '/content/drive/MyDrive/dataset/#bench_marks/VisDrone2019' # put your path to visdrone2019



In [ ]:
# directory names for train, val and test set 
train_dir_name = 'VisDrone2019-DET-train'
val_dir_name = 'VisDrone2019-DET-val'
test_dir_name = 'VisDrone2019-DET-test-dev' # sample test images with ground truth 

train_dir = os.path.join(data_root_dir, train_dir_name)
val_dir = os.path.join(data_root_dir, val_dir_name)
test_dir = os.path.join(data_root_dir, test_dir_name)

train_img_list = glob(os.path.join(train_dir, 'images', "*.jpg"))
val_img_list = glob(os.path.join(val_dir, 'images', "*.jpg"))
test_img_list = glob(os.path.join(test_dir, 'images', "*.jpg"))

train_ann_list = glob(os.path.join(train_dir, 'annotations', "*.txt"))
val_ann_list = glob(os.path.join(val_dir, 'annotations', "*.txt"))
test_ann_list = glob(os.path.join(test_dir, 'annotations', "*.txt"))

print("Num of train images: {} ".format(len(train_img_list)))
print("Num of validation images: {} ".format(len(val_img_list)))
print("Num of test-dev images: {} ".format(len(test_img_list)))

In [ ]:
# Let's take a look at the dataset image
import mmcv
import matplotlib.pyplot as plt
import random 

# img_idx = 0 # set specific image index 
img_idx = random.randint(0, len(train_img_list)-1)

img = mmcv.imread(train_img_list[img_idx])
plt.figure(figsize=(15, 10))
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

According to the VisDrone's documentation, the first four columns includes bbox information, and the 5th to 8th columns indicates score, object category, truncation and occlusion respectively. We need to read annotations of each image and convert them into middle format MMDetection accept is as below:

```python
[ <bbox_left>,<bbox_top>,<bbox_width>,<bbox_height>,<score>,<object_category>,<truncation>,<occlusion> ]
```

In [ ]:
# Let's take a look at the dataset annotation format

# img_idx = 0 # set specific image index 
ann_idx = random.randint(0, len(train_img_list)-1)

ann_txt  = mmcv.list_from_file(train_ann_list[ann_idx]) # read a text file to list 

# please refer to annotation style at VisDrone2019 hompage : "http://aiskyeye.com/evaluate/results-format_2021/"

print('<bbox_left>,<bbox_top>,<bbox_width>,<bbox_height>,<score>,<object_category>,<truncation>,<occlusion>')
print(ann_txt[1])

In [ ]:
import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from tqdm import tqdm

@DATASETS.register_module()
class VisDrone2019Dataset(CustomDataset):

    """
    The object category indicates the type of annotated object, 
    (i.e., ignored regions  (0), pedestrian (1), people (2), bicycle (3), 
            car             (4), van        (5), truck  (6), tricycle (7),
            awning-tricycle (8), bus        (9), motor (10), others (11))
    """

    CLASSES = ('pedestrian', 'people', 'bicycle'
               'car', 'van', 'truck', 'tricycle', 'awning-tricycle', 
               'bus', 'motor', 'others') # exclude ignored regions

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}

        data_infos = []
        image_list = glob(osp.join(self.data_root, self.img_prefix, '*jpg'))
        ann_dir = osp.join(self.data_root, self.ann_file)
        

        for image_dir in tqdm(image_list,  desc="Load annotations") : 
            image = mmcv.imread(image_dir)
            height, width = image.shape[:2]
            filename = image_dir.split('/')[-1]

            data_info = dict(filename=filename, width=width, height=height)

            # load annotations
            ann_txt = mmcv.list_from_file(osp.join(ann_dir, filename.replace('jpg', 'txt')))

            contents = [line.strip().split(',') for line in ann_txt]
            bbox_labels = [x[5] for x in contents]

            bboxes = []
            for content in contents : 
                x, y, w, h = content[0:4]
                x, y, w, h = float(x), float(y), float(w), float(h)
                bboxes.append([x, y, x+w, y+h])

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            # filter 'DontCare'
            for bbox_label, bbox in zip(bbox_labels, bboxes):
                if bbox_label == '0':
                    gt_labels_ignore.append(-1)
                    gt_bboxes_ignore.append(bbox)
                else:
                    gt_labels.append(int(bbox_label)-1)
                    gt_bboxes.append(bbox)
                    
            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.long),
                bboxes_ignore=np.array(gt_bboxes_ignore,
                                       dtype=np.float32).reshape(-1, 4),
                labels_ignore=np.array(gt_labels_ignore, dtype=np.long))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos      

### IoU-Deviation Curve and NWD-Deviation Curve Comparison 

In [ ]:
from mmdet.core.bbox.iou_calculators.iou2d_calculator import bbox_overlaps


def collect_iou_calculation(scales,dev_range, center_x, center_y, bbox1_size_factor, bbox2_size_factor):
    bboxes1 = []
    bboxes2 = []
    overlap_cal_results = []
    for scale_idx, scale in enumerate(scales):
        width = bbox1_size_factor*scale 
        height = bbox1_size_factor*scale 
        x1, y1, x2, y2 = int(center_x-width/2), int(center_y-height/2), int(center_x+width/2), int(center_y+height/2)
        bbox1 = [x1, y1, x2, y2]

        bboxes1.append(bbox1)
        bboxes2.append([])

        for deviation in np.arange(dev_range[0], dev_range[-1]):
            width = bbox2_size_factor*scale 
            height = bbox2_size_factor*scale 
            x1, y1, x2, y2 = int(center_x-width/2), int(center_y-height/2), int(center_x+width/2), int(center_y+height/2)
            bbox2 =  [x1+deviation, y1+deviation, x2+deviation, y2+deviation]
            bboxes2[scale_idx].append(bbox2)
        
        bbox1_tensor = torch.FloatTensor([bbox1])
        bboxes2_tensor = torch.FloatTensor(bboxes2[scale_idx])
        

        overlap_cal_results.append(bbox_overlaps(bbox1_tensor, bboxes2_tensor)[0])

    return overlap_cal_results

In [ ]:
from scipy.stats import wasserstein_distance
from numpy.linalg import norm 


def collect_nwd_calculation(scales, dev_range, center_x, center_y, bbox1_size_factor, bbox2_size_factor, c=1):
    bboxes1 = []
    bboxes2 = []
    overlap_cal_results = []
    for scale_idx, scale in enumerate(scales):
        overlap_cal_result = []
        width = bbox1_size_factor*scale 
        height = bbox1_size_factor*scale 
        bbox1 = [center_x, center_y, width/2, height/2]

        for deviation in np.arange(dev_range[0], dev_range[-1]):
            width = bbox2_size_factor*scale 
            height = bbox2_size_factor*scale 
            bbox2 =  [center_x+deviation, center_y+deviation, width/2, height/2]
            w_dist = wasserstein_distance(bbox1, bbox2)
            # w_dist = norm(np.array([bbox1, bbox2]).T, ord=2)
            nwd = np.exp(-np.sqrt(w_dist**2)/c)
            overlap_cal_result.append(nwd)


        overlap_cal_results.append(overlap_cal_result)

    return overlap_cal_results

In [ ]:
import matplotlib.pyplot as plt 

center_x, center_y = 256, 256
bbox1_size_factor = 1
bbox2_size_factor = [1, 0.5]
scales = [4, 12, 32, 48]
dev_range = [-30, 30]

overlap_cal_results_0 = collect_iou_calculation(scales,dev_range, center_x, center_y, bbox1_size_factor, bbox2_size_factor[0])
overlap_cal_results_1 = collect_iou_calculation(scales,dev_range, center_x, center_y, bbox1_size_factor, bbox2_size_factor[1])

fig, axs = plt.subplots(1, 2, figsize=(16, 6))

for result in overlap_cal_results_0 : 
    axs[0].plot(np.arange(dev_range[0], dev_range[1]), result, 'o--')
    axs[0].set_xlabel('Deviation', fontsize=15)
    axs[0].set_ylabel('IoU', fontsize=15)
    axs[0].set_title('IoU-Deviation Curve : same size', fontsize=15)
    axs[0].set_ylim(0, 1.0)


for result in overlap_cal_results_1 : 
    axs[1].plot(np.arange(dev_range[0], dev_range[1]), result, 'o--')
    axs[1].set_xlabel('Deviation', fontsize=15)
    axs[1].set_ylabel('IoU', fontsize=15)
    axs[1].set_title('IoU-Deviation Curve : 1/2 size', fontsize=15)
    axs[1].set_ylim(0, 0.5)

In [ ]:
import matplotlib.pyplot as plt 

center_x, center_y = 256, 256
bbox1_size_factor = 1
bbox2_size_factor = [1, 0.5]
scales = [4, 12, 32, 48]
dev_range = [-30, 30]
c = 4
overlap_cal_results_0 = collect_nwd_calculation(scales,dev_range, center_x, center_y, bbox1_size_factor, bbox2_size_factor[0], c=c)
overlap_cal_results_1 = collect_nwd_calculation(scales,dev_range, center_x, center_y, bbox1_size_factor, bbox2_size_factor[1], c=c)

fig, axs = plt.subplots(1, 2, figsize=(16, 6))

for result in overlap_cal_results_0 : 
    axs[0].plot(np.arange(dev_range[0], dev_range[1]), result, 'o--')
    axs[0].set_xlabel('Deviation', fontsize=15)
    axs[0].set_ylabel('NWD', fontsize=15)
    axs[0].set_title('NWD-Deviation Curve : same size', fontsize=15)
    axs[0].set_ylim(0, 1.0)


for result in overlap_cal_results_1 : 
    axs[1].plot(np.arange(dev_range[0], dev_range[1]), result, 'o--')
    axs[1].set_xlabel('Deviation', fontsize=15)
    axs[1].set_ylabel('NWD', fontsize=15)
    axs[1].set_title('NWD-Deviation Curve : 1/2 size', fontsize=15)
    axs[1].set_ylim(0, 1.0)

In [ ]:
# NWD-based label assginment 

# mmdetection/mmdet/core/bbox/assigners/

In [ ]:
# NWD-based NMS 
# https://github.com/open-mmlab/mmcv/blob/master/mmcv/ops/nms.py



In [ ]:
# NWD-based regression loss 
# mmdetection/mmdet/models/losses/

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a detector using a pre-trained detector.

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

In [ ]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VisDrone2019Dataset'
cfg.data_root = data_root_dir

cfg.data.test.type = 'VisDrone2019Dataset'
cfg.data.test.data_root = '{}/{}/'.format(data_root_dir, test_dir_name) 
cfg.data.test.ann_file = 'annotations'
cfg.data.test.img_prefix = 'images'

cfg.data.train.type = 'VisDrone2019Dataset'
cfg.data.train.data_root = '{}/{}/'.format(data_root_dir, train_dir_name) 
cfg.data.train.ann_file = 'annotations'
cfg.data.train.img_prefix = 'images'

cfg.data.val.type = 'VisDrone2019Dataset'
cfg.data.val.data_root = '{}/{}/'.format(data_root_dir, val_dir_name) 
cfg.data.val.ann_file = 'annotations'
cfg.data.val.img_prefix = 'images'

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 11 # exclude ignored regions

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 500

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


### Train a new detector

Finally, lets initialize the dataset and detector, then train a new detector!

In [ ]:
# download checkpoint file 
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
      -O checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)] # should take a long time if google drive is used....

In [ ]:
# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)
